In [1]:
import geopandas as gpd
shapefile = './data/countries_110m/ne_110m_admin_0_countries.shp'
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
#Rename columns.
gdf.columns = ['country_name', 'country_code', 'geometry']
gdf.head()

country_name country_code  \
0                         Fiji          FJI   
1  United Republic of Tanzania          TZA   
2               Western Sahara          SAH   
3                       Canada          CAN   
4     United States of America          USA   

                                            geometry  
0  (POLYGON ((180 -16.06713266364245, 180 -16.555...  
1  POLYGON ((33.90371119710453 -0.950000000000000...  
2  POLYGON ((-8.665589565454809 27.65642588959236...  
3  (POLYGON ((-122.84 49.00000000000011, -122.974...  
4  (POLYGON ((-122.84 49.00000000000011, -120 49....

In [2]:
print(gdf[gdf['country_name'] == 'Antarctica'])
#Drop row corresponding to 'Antarctica'
gdf = gdf.drop(gdf.index[159])

    country_name country_code  \
159   Antarctica          ATA   

                                              geometry  
159  (POLYGON ((-48.66061601418252 -78.047018731598...  


In [3]:
grid_crs=gdf.crs
grid_crs


{'init': 'epsg:4326'}

In [4]:
import pandas as pd
import numpy as np
import requests
pathtoCSV=r'https://docs.google.com/spreadsheets/d/e/2PACX-1vR3adItsg2hhUehyNsKvr0X-ipi6HBw6pgp6ToNevHHgTYe8EuxENlmQehi1iVEEnrqPk_agdBNTLtU/pub?output=csv'
#Read csv file using pandas
df1 = pd.read_csv(pathtoCSV, encoding='utf8',names = ['continent', 'country/territory', 'confirmed_cases', 'deaths'], index_col=False,skiprows=1,skipfooter=1)
df1['scaled_confirmed_cases'] = np.log(df1['confirmed_cases']+1)*1.5
df1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


continent                           country/territory  confirmed_cases  \
0       Asia                                       China            74258   
1       Asia                                   Singapore               81   
2       Asia                                       Japan               66   
3       Asia                           Republic of Korea               46   
4       Asia                                    Thailand               35   
5       Asia                                      Taiwan               22   
6       Asia                                    Malaysia               22   
7       Asia                                     Vietnam               16   
8       Asia                        United Arab Emirates                9   
9       Asia                                       India                3   
10      Asia                                 Philippines                3   
11      Asia                                       Nepal                1   
12      Asia                                   Sri Lanka                1   
13      Asia                                    Cambodia                1   
14     Other  Cases on an international conveyance Japan              542   
15    Europe                                     Germany               16   
16    Europe                                      France               12   
17    Europe                              United Kingdom                9   
18    Europe                                       Italy                3   
19    Europe                                       Spain                2   
20    Europe                                      Russia                2   
21    Europe                                      Sweden                1   
22    Europe                                     Finland                1   
23    Europe                                     Belgium                1   
24   America                    United States of America               15   
25   America                                      Canada                8   
26   Oceania                                   Australia               15   
27    Africa                                       Egypt                1   

    deaths  scaled_confirmed_cases  
0     2008               16.822971  
1        0                6.610079  
2        1                6.307039  
3        0                5.775221  
4        0                5.375278  
5        1                4.703241  
6        0                4.703241  
7        0                4.249820  
8        0                3.453878  
9        0                2.079442  
10       1                2.079442  
11       0                1.039721  
12       0                1.039721  
13       0                1.039721  
14       0                9.445664  
15       0                4.249820  
16       1                3.847424  
17       0                3.453878  
18       0                2.079442  
19       0                1.647918  
20       0                1.647918  
21       0                1.039721  
22       0                1.039721  
23       0                1.039721  
24       0                4.158883  
25       0                3.295837  
26       0                4.158883  
27       0                1.039721

In [5]:
datafile_2='./data/country_geocodes.csv'
df2 = pd.read_csv(datafile_2, names = ['country', 'latitude', 'longitude'], skiprows = 1)
df2.head()

country   latitude  longitude
0  Afghanistan  33.939110  67.709953
1      Albania  41.153332  20.168331
2      Algeria  28.033886   1.659626
3      Andorra  42.506285   1.521801
4       Angola -11.202692  17.873887

In [6]:
points = pd.merge(df1, df2, left_on='country/territory',right_on='country', how='left')
points_df = points.drop(['continent','country'],axis=1)
points_df.head()



country/territory  confirmed_cases  deaths  scaled_confirmed_cases  \
0              China            74258    2008               16.822971   
1          Singapore               81       0                6.610079   
2              Japan               66       1                6.307039   
3  Republic of Korea               46       0                5.775221   
4           Thailand               35       0                5.375278   

    latitude   longitude  
0  35.861660  104.195397  
1   1.352083  103.819836  
2  36.204824  138.252924  
3  35.907757  127.766922  
4  15.870032  100.992541

In [7]:
from shapely.geometry import Point
# creating a geometry column 
geometry_centroids = [Point(xy) for xy in zip(points_df['longitude'], points_df['latitude'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Creating a Geographic data frame 
points_gdf = gpd.GeoDataFrame(points_df, crs=crs, geometry=geometry_centroids)
points_gdf.head()

country/territory  confirmed_cases  deaths  scaled_confirmed_cases  \
0              China            74258    2008               16.822971   
1          Singapore               81       0                6.610079   
2              Japan               66       1                6.307039   
3  Republic of Korea               46       0                5.775221   
4           Thailand               35       0                5.375278   

    latitude   longitude                      geometry  
0  35.861660  104.195397   POINT (104.195397 35.86166)  
1   1.352083  103.819836   POINT (103.819836 1.352083)  
2  36.204824  138.252924  POINT (138.252924 36.204824)  
3  35.907757  127.766922  POINT (127.766922 35.907757)  
4  15.870032  100.992541  POINT (100.992541 15.870032)

In [12]:
from geopandas.tools import sjoin
joined = gpd.sjoin(gdf, points_gdf, how="left", op='intersects')
joined.head()






country_name country_code  \
0                         Fiji          FJI   
1  United Republic of Tanzania          TZA   
2               Western Sahara          SAH   
3                       Canada          CAN   
4     United States of America          USA   

                                            geometry  index_right  \
0  (POLYGON ((180 -16.06713266364245, 180 -16.555...          NaN   
1  POLYGON ((33.90371119710453 -0.950000000000000...          NaN   
2  POLYGON ((-8.665589565454809 27.65642588959236...          NaN   
3  (POLYGON ((-122.84 49.00000000000011, -122.974...         25.0   
4  (POLYGON ((-122.84 49.00000000000011, -120 49....         24.0   

          country/territory  confirmed_cases  deaths  scaled_confirmed_cases  \
0                       NaN              NaN     NaN                     NaN   
1                       NaN              NaN     NaN                     NaN   
2                       NaN              NaN     NaN                     NaN   
3                    Canada              8.0     0.0                3.295837   
4  United States of America             15.0     0.0                4.158883   

    latitude   longitude  
0        NaN         NaN  
1        NaN         NaN  
2        NaN         NaN  
3  56.130366 -106.346771  
4  37.090240  -95.712891

In [9]:
joined.fillna("None", inplace=True)

In [10]:
import json
#Read data to json.
joined_json = json.loads(joined.to_json())
#Convert to String like object.
json_data = json.dumps(joined_json)

In [11]:
from bokeh.io import curdoc, output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource
from bokeh.models import HoverTool
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)


#Create figure object.
p = figure(title = 'Worldwide distribution of nCov cases, 2020',plot_height = 600 , plot_width = 1050, 
           background_fill_color="#2b8cbe", background_fill_alpha=0.4)
p.title.text_font = "helvetica"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
patch=p.patches('xs','ys', source = geosource,fill_color = '#fff7bc',
          line_color = '#d95f0e', line_width = 0.35, fill_alpha = 1, 
                hover_fill_color="#fec44f")
point=p.circle('longitude','latitude',source=geosource, radius='scaled_confirmed_cases',
               fill_color="firebrick", alpha=0.7)
p.add_tools(HoverTool(tooltips=[('Country','@country_name'),('Confirmed Cases','@confirmed_cases'), 
                                ('Deaths','@deaths')], renderers=[patch]))


#Display figure inline in Jupyter Notebook.
output_notebook()
curdoc().add_root(p)
#Display figure.
show(p)

Loading BokehJS ...